## https://wikidocs.net/22891

# 1. Spam Detection

In [ ]:
import pandas as pd
import numpy as np
from sklearn import metrics
import tensorflow as tf
from keras import optimizers
import matplotlib.pyplot as plt
import urllib.request
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
#  https://www.kaggle.com/uciml/sms-spam-collection-dataset
data = pd.read_csv(r'spam.csv', encoding='latin1')
print('sample number:',len(data))

In [ ]:
data[:5]

In [ ]:
del data['Unnamed: 2']
del data['Unnamed: 3']
del data['Unnamed: 4']
data['v1'] = data['v1'].replace(['ham','spam'],[0,1])
data[:5]

In [ ]:
data.info()

In [ ]:
# missing?
data.isnull().values.any()

In [ ]:
# unique?
data['v2'].nunique()

In [ ]:
# delete duplicate
data.drop_duplicates(subset=['v2'], inplace=True)
data.info()

In [ ]:
numlist = list(range(len(data)))
data = data.set_index(pd.Index(numlist))
data = data[:1000]
data

In [ ]:
data['v1'].value_counts()

In [ ]:
data.drop(data[:824][data['v1'][:824] == 0].index, inplace=True)
numlist = list(range(len(data)))
data = data.set_index(pd.Index(numlist))
data

In [ ]:
data['v1'].value_counts()

In [ ]:
X = data['v2']
y = data['v1']
y = y.astype(float)

In [ ]:
print(X)

In [ ]:
# integer encoding
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X_encoded = tokenizer.texts_to_sequences(X)

In [ ]:
# index (more frequent, smaller number given)
word_to_index = tokenizer.word_index
# print(len(word_to_index), word_to_index)

In [ ]:
# word group for padding
vocab_size = len(word_to_index) + 1
print(vocab_size)

In [ ]:
# the longest email
maxlen = 0
for i in range(len(X_encoded)):
    if len(X_encoded[i]) >= maxlen:
        maxlen = len(X_encoded[i])
print(maxlen)

In [ ]:
# padding
max_len = maxlen
X_padded = pad_sequences(X_encoded, maxlen = max_len)
X_padded.shape

In [ ]:
X = X_padded
X

In [ ]:
y_hard = pd.DataFrame(y)
y_hard

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

epochs = 10
batch = 64

from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, LSTM, GRU
from tensorflow.keras.models import Sequential
from keras import optimizers

embedding_dim = 32
hidden_units = 32

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
def create_model():
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim))
    # model.add(SimpleRNN(hidden_units))
    model.add(LSTM(hidden_units))
    # model.add(GRU(hidden_units))
    model.add(Dense(1, activation='sigmoid'))    
    return model

# 1-0. Generating Prob_labels

In [ ]:
gen_model = create_model()   
gen_model.compile(loss='BinaryCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.001), metrics=['accuracy'])
history = gen_model.fit(X, y_hard, validation_split=0.2, epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
plt.plot(history.history['loss'], label='loss')
plt.ylim([0, 1])
plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
plt.ylabel('Loss',fontweight="bold",fontsize = 15)
plt.title("Cost Function",fontweight="bold",fontsize = 20)
plt.legend()
plt.show()
result = gen_model.predict(X, verbose=0)
prob_label = list(result.reshape(len(X),))
prob_label

# 1-1. Focal(Hard) and SLS(Hard/diverse alphas)_option#1

In [ ]:
for t in range(10):    # 10 times repeat    
    res = pd.DataFrame({'Focal':[0, 0, 0, 0, 0]}, index = ['Acc','Pre','Rec','F1','R-AUC']) 
    # Focal
    print('#'*50,'Focal','#'*50)
    list_acc = []
    list_pre = []
    list_rec = []
    list_f1 = []
    list_rauc = []   
    focal_model = create_model()   

    n_iter = 0
    for train_index, test_index in skf.split(X, y_hard):  # straticiation by y_hard(binary label)
        n_iter += 1
        X_train = X[train_index]
        y_train= y_hard.iloc[train_index]
        if n_iter == 1:
            print(y_train.value_counts())
        X_test = X[test_index]
        y_test= y_hard.iloc[test_index]
#         print('#'*10,'{0}th CV'.format(n_iter),'#'*10)
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        y_train = y_train.astype(float)
        X_test = np.array(X_test)
        y_test = np.array(y_test)
        y_test = y_test.astype(float)

        focal_model.compile(loss='BinaryFocalCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.001), metrics=['accuracy'])
        history = focal_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#         plt.plot(history.history['loss'], label='loss')
#         plt.ylim([0, 1])
#         plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#         plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#         plt.title("Cost Function",fontweight="bold",fontsize = 20)
#         plt.legend()
#         plt.show()
        predicted = np.round(focal_model.predict(X_test, verbose=0))
        list_acc.append(metrics.accuracy_score(y_test, predicted))
        list_pre.append(metrics.precision_score(y_test, predicted))
        list_rec.append(metrics.recall_score(y_test, predicted))
        list_f1.append(metrics.f1_score(y_test, predicted))
        list_rauc.append(metrics.roc_auc_score(y_test, predicted))
    res['Focal'] = [np.mean(list_acc), np.mean(list_pre), np.mean(list_rec), np.mean(list_f1), np.mean(list_rauc)]
    print([np.mean(list_acc), np.mean(list_pre), np.mean(list_rec), np.mean(list_f1), np.mean(list_rauc)])
    
    B = [0.00, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]  # SLS with LS
    c = 0.1  # criterion decides easy/hard
    for b in B:
        print('#'*50,'SLS',b,'#'*50)
        y_005 = []
        for i in range(len(y_hard)):
            if list(y_hard['v1'])[i] == 0:
                if prob_label[i] <= c:
                    y_005.append(b)  # easy sample
                else:
                    y_005.append(0) # (or 0-b) hard sample
            if list(y_hard['v1'])[i] == 1:
                if prob_label[i] >= 1-c:
                    y_005.append(1-b)  # easy sample
                else:
                    y_005.append(1) # (or 1+b) hard sample
        y_005 = pd.DataFrame(y_005)     

        bce005_acc = []
        bce005_pre = []
        bce005_rec = []
        bce005_f1 = []
        bce005_rocauc = []
        model_005 = create_model()
#             early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        n_iter = 0
        for train_index, test_index in skf.split(X, y_hard):  # straticiation by y_hard(binary label)
            n_iter += 1
            X_train = X[train_index]
            y_005_train= y_005.iloc[train_index]
            if n_iter == 1:
                print(y_005_train.value_counts())
            X_test = X[test_index]
            y_test= y_hard.iloc[test_index]  # test with real(actual) label y
#                 print('#'*10,'{0}th CV'.format(n_iter),'#'*10)
            X_train = np.array(X_train)
            y_005_train = np.array(y_005_train)
            y_005_train = y_005_train.astype(float)
            X_test = np.array(X_test)
            y_test = np.array(y_test)
            y_test = y_test.astype(float)

            # MLP_BCE(y_005)
            model_005.compile(loss='BinaryCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.001), metrics=['accuracy'])
            history = model_005.fit(X_train, y_005_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#             plt.plot(history.history['loss'], label='loss')
#             plt.ylim([0, 1])
#             plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#             plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#             plt.title("Cost Function",fontweight="bold",fontsize = 20)
#             plt.legend()
#             plt.show()
            predicted = np.round(model_005.predict(X_test, verbose=0))
            bce005_acc.append(metrics.accuracy_score(y_test, predicted))
            bce005_pre.append(metrics.precision_score(y_test, predicted))
            bce005_rec.append(metrics.recall_score(y_test, predicted))
            bce005_f1.append(metrics.f1_score(y_test, predicted))
            bce005_rocauc.append(metrics.roc_auc_score(y_test, predicted))
        res['SLS({})'.format(b)] = [np.mean(bce005_acc), np.mean(bce005_pre), np.mean(bce005_rec), np.mean(bce005_f1), np.mean(bce005_rocauc)]
        print([np.mean(bce005_acc), np.mean(bce005_pre), np.mean(bce005_rec), np.mean(bce005_f1), np.mean(bce005_rocauc)])        
    res.to_csv("RNN_SPAM_5CV(SLS_opt#1_c0.1).csv", mode = 'a', float_format='%.4g')

In [ ]:
res = pd.read_csv("RNN_SPAM_5CV(SLS_opt#1_c0.1).csv")
res = res.dropna(axis=0)
res = res.rename(columns={'SLS(0.0)':'Hard'})
df_acc = res[res.iloc[:,0] == 'Acc']
df_acc = df_acc.reset_index(drop=True)
df_acc = df_acc.iloc[:,1:].astype(float)
col_name = df_acc.columns
ave = []
std = []
for i in range(12):
    ave.append(np.mean(list(df_acc.iloc[:,i])))
    std.append(np.std(list(df_acc.iloc[:,i])))
final = pd.DataFrame(ave, index=col_name, columns=["mean"])
final['std'] = std
final.sort_values("mean", ascending=False)

In [ ]:
len(df_acc)

# 2. Reuters News

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.sequence import pad_sequences
import seaborn as sns
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import reuters

In [ ]:
vocab_size = 1000

(XX, yy), (X_dummy, y_dummy) = reuters.load_data(num_words=vocab_size, test_split=0)

print(len(XX))
print(len(X_dummy))
num_classes = len(set(yy))
print(num_classes)

In [ ]:
print(XX[0])
print(yy[0])

In [ ]:
word_to_index = reuters.get_word_index()
# print(len(word_to_index), word_to_index)

In [ ]:
# word group for padding
# vocab_size = len(word_to_index) + 3
print(vocab_size)

In [ ]:
# the longest news
maxlen = 0
for i in range(len(XX)):
    if len(XX[i]) >= maxlen:
        maxlen = len(XX[i])
        
# maxlen = 100
print(maxlen)

In [ ]:
# padding
max_len = maxlen
X_padded = pad_sequences(XX, maxlen = max_len)
X_padded.shape

In [ ]:
pd.Series(yy).value_counts()

In [ ]:
# Picking only label 3->'0' & 1->'1'
idx_4 = []
idx_3 = []
for i in range(len(yy)):
    if list(yy)[i] == 4:
        idx_4.append(i)
    if list(yy)[i] == 3:
        idx_3.append(i)
print(len(idx_4), len(idx_3))

In [ ]:
idx = idx_4[:150] + idx_3[:150]
idx.sort()

In [ ]:
X = []
y = []
for i in idx:
    X.append(X_padded[i])
    y.append(yy[i])
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

In [ ]:
pd.Series(y).value_counts()

In [ ]:
y_hard = [0.00 if x==3 else x for x in y]
y_hard = [1.00 if x==4 else x for x in y_hard]
pd.Series(y_hard).value_counts()

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

epochs = 10
batch = 64

from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, LSTM, GRU
from tensorflow.keras.models import Sequential
from keras import optimizers

embedding_dim = 32
hidden_units = 32

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
def create_model():
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim))
    # model.add(SimpleRNN(hidden_units))
    model.add(LSTM(hidden_units))
    # model.add(GRU(hidden_units))
    model.add(Dense(1, activation='sigmoid'))    
    return model

In [ ]:
X

In [ ]:
y_hard = pd.DataFrame(y_hard)
y_hard

# 2-0. Generating Prob_labels

In [ ]:
gen_model = create_model()   
gen_model.compile(loss='BinaryCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.005), metrics=['accuracy'])
history = gen_model.fit(X, y_hard, validation_split=0.2, epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
plt.plot(history.history['loss'], label='loss')
plt.ylim([0, 1])
plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
plt.ylabel('Loss',fontweight="bold",fontsize = 15)
plt.title("Cost Function",fontweight="bold",fontsize = 20)
plt.legend()
plt.show()
result = gen_model.predict(X, verbose=0)
prob_label = list(result.reshape(len(X),))
prob_label

# 2-1. Focal(Hard) and SLS(Hard/diverse alphas)_option#1

In [ ]:
for t in range(10):    # 10 times repeat    
    res = pd.DataFrame({'Focal':[0, 0, 0, 0, 0]}, index = ['Acc','Pre','Rec','F1','R-AUC']) 
    # Focal
    print('#'*50,'Focal','#'*50)
    list_acc = []
    list_pre = []
    list_rec = []
    list_f1 = []
    list_rauc = []   
    focal_model = create_model()   

    n_iter = 0
    for train_index, test_index in skf.split(X, y_hard):  # straticiation by y_hard(binary label)
        n_iter += 1
        X_train = X[train_index]
        y_train= y_hard.iloc[train_index]
        if n_iter == 1:
            print(y_train.value_counts())
        X_test = X[test_index]
        y_test= y_hard.iloc[test_index]
#         print('#'*10,'{0}th CV'.format(n_iter),'#'*10)
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        y_train = y_train.astype(float)
        X_test = np.array(X_test)
        y_test = np.array(y_test)
        y_test = y_test.astype(float)

        focal_model.compile(loss='BinaryFocalCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.005), metrics=['accuracy'])
        history = focal_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#         plt.plot(history.history['loss'], label='loss')
#         plt.ylim([0, 1])
#         plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#         plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#         plt.title("Cost Function",fontweight="bold",fontsize = 20)
#         plt.legend()
#         plt.show()
        predicted = np.round(focal_model.predict(X_test, verbose=0))
        list_acc.append(metrics.accuracy_score(y_test, predicted))
        list_pre.append(metrics.precision_score(y_test, predicted))
        list_rec.append(metrics.recall_score(y_test, predicted))
        list_f1.append(metrics.f1_score(y_test, predicted))
        list_rauc.append(metrics.roc_auc_score(y_test, predicted))
    res['Focal'] = [np.mean(list_acc), np.mean(list_pre), np.mean(list_rec), np.mean(list_f1), np.mean(list_rauc)]
    print([np.mean(list_acc), np.mean(list_pre), np.mean(list_rec), np.mean(list_f1), np.mean(list_rauc)])
    
    B = [0.00, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]  # SLS with LS
    c = 0.1  # criterion decides easy/hard
    for b in B:
        print('#'*50,'SLS',b,'#'*50)
        y_005 = []
        for i in range(len(y_hard)):
            if list(y_hard[0])[i] == 0:
                if prob_label[i] <= c:
                    y_005.append(b)  # easy sample
                else:
                    y_005.append(0) # (or 0-b) hard sample
            if list(y_hard[0])[i] == 1:
                if prob_label[i] >= 1-c:
                    y_005.append(1-b)  # easy sample
                else:
                    y_005.append(1) # (or 1+b) hard sample
        y_005 = pd.DataFrame(y_005)     

        bce005_acc = []
        bce005_pre = []
        bce005_rec = []
        bce005_f1 = []
        bce005_rocauc = []
        model_005 = create_model()
#             early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        n_iter = 0
        for train_index, test_index in skf.split(X, y_hard):  # straticiation by y_hard(binary label)
            n_iter += 1
            X_train = X[train_index]
            y_005_train= y_005.iloc[train_index]
            if n_iter == 1:
                print(y_005_train.value_counts())
            X_test = X[test_index]
            y_test= y_hard.iloc[test_index]  # test with real(actual) label y
#                 print('#'*10,'{0}th CV'.format(n_iter),'#'*10)
            X_train = np.array(X_train)
            y_005_train = np.array(y_005_train)
            y_005_train = y_005_train.astype(float)
            X_test = np.array(X_test)
            y_test = np.array(y_test)
            y_test = y_test.astype(float)

            # MLP_BCE(y_005)
            model_005.compile(loss='BinaryCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.005), metrics=['accuracy'])
            history = model_005.fit(X_train, y_005_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#             plt.plot(history.history['loss'], label='loss')
#             plt.ylim([0, 1])
#             plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#             plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#             plt.title("Cost Function",fontweight="bold",fontsize = 20)
#             plt.legend()
#             plt.show()
            predicted = np.round(model_005.predict(X_test, verbose=0))
            bce005_acc.append(metrics.accuracy_score(y_test, predicted))
            bce005_pre.append(metrics.precision_score(y_test, predicted))
            bce005_rec.append(metrics.recall_score(y_test, predicted))
            bce005_f1.append(metrics.f1_score(y_test, predicted))
            bce005_rocauc.append(metrics.roc_auc_score(y_test, predicted))
        res['SLS({})'.format(b)] = [np.mean(bce005_acc), np.mean(bce005_pre), np.mean(bce005_rec), np.mean(bce005_f1), np.mean(bce005_rocauc)]
        print([np.mean(bce005_acc), np.mean(bce005_pre), np.mean(bce005_rec), np.mean(bce005_f1), np.mean(bce005_rocauc)])        
    res.to_csv("RNN_RNEWS_5CV(SLS_opt#1_c0.1).csv", mode = 'a', float_format='%.4g')

In [ ]:
res = pd.read_csv("RNN_RNEWS_5CV(SLS_opt#1_c0.1).csv")
res = res.dropna(axis=0)
res = res.rename(columns={'SLS(0.0)':'Hard'})
df_acc = res[res.iloc[:,0] == 'Acc']
df_acc = df_acc.reset_index(drop=True)
df_acc = df_acc.iloc[:,1:].astype(float)
col_name = df_acc.columns
ave = []
std = []
for i in range(12):
    ave.append(np.mean(list(df_acc.iloc[:,i])))
    std.append(np.std(list(df_acc.iloc[:,i])))
final = pd.DataFrame(ave, index=col_name, columns=["mean"])
final['std'] = std
final.sort_values("mean", ascending=False)

In [ ]:
len(df_acc)

# IMDB Sentiment Analysis

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.datasets import imdb

In [ ]:
vocab_size = 1000
max_len = 100

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

In [ ]:
print(len(X_train), len(y_train))

In [ ]:
pd.Series(y_train).value_counts()

In [ ]:
# Making lists of index
idx_0 = []
idx_1 = []
for i in range(len(y_train)):
    if list(y_train)[i] == 0:
        idx_0.append(i)
    if list(y_train)[i] == 1:
        idx_1.append(i)
print(len(idx_0), len(idx_1))

In [ ]:
idx = idx_0[:150] + idx_1[:150]
idx.sort()
# print(len(idx), idx)

In [ ]:
X = []
y = []
for i in idx:
    X.append(X_train[i])
    y.append(y_train[i])
X = np.array(X)
y = np.array(y)
print(X.shape, y.shape)

In [ ]:
print(pd.Series(y).value_counts())

In [ ]:
word_to_index = imdb.get_word_index()
# print(len(word_to_index), word_to_index)

In [ ]:
# padding
X_padded = pad_sequences(X, maxlen = max_len)
X_padded.shape

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state = 2)

epochs = 10
batch = 64

from tensorflow.keras.layers import SimpleRNN, Embedding, Dense, LSTM, GRU
from tensorflow.keras.models import Sequential
from keras import optimizers

embedding_dim = 32
hidden_units = 32

from keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
def create_model():
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim))
    # model.add(SimpleRNN(hidden_units))
    model.add(LSTM(hidden_units))
    # model.add(GRU(hidden_units))
    model.add(Dense(1, activation='sigmoid'))    
    return model

In [ ]:
X = X_padded
X

In [ ]:
y_hard = pd.DataFrame(y_hard)
y_hard

# 3-0. Generating Prob_labels

In [ ]:
gen_model = create_model()   
gen_model.compile(loss='BinaryCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.005), metrics=['accuracy'])
history = gen_model.fit(X, y_hard, validation_split=0.2, epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
plt.plot(history.history['loss'], label='loss')
plt.ylim([0, 1])
plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
plt.ylabel('Loss',fontweight="bold",fontsize = 15)
plt.title("Cost Function",fontweight="bold",fontsize = 20)
plt.legend()
plt.show()
result = gen_model.predict(X, verbose=0)
prob_label = list(result.reshape(len(X),))
prob_label

# 3-1. Focal(Hard) and SLS(Hard/diverse alphas)_option#1

In [ ]:
for t in range(10):    # 10 times repeat    
    res = pd.DataFrame({'Focal':[0, 0, 0, 0, 0]}, index = ['Acc','Pre','Rec','F1','R-AUC']) 
    # Focal
    print('#'*50,'Focal','#'*50)
    list_acc = []
    list_pre = []
    list_rec = []
    list_f1 = []
    list_rauc = []   
    focal_model = create_model()   

    n_iter = 0
    for train_index, test_index in skf.split(X, y_hard):  # straticiation by y_hard(binary label)
        n_iter += 1
        X_train = X[train_index]
        y_train= y_hard.iloc[train_index]
        if n_iter == 1:
            print(y_train.value_counts())
        X_test = X[test_index]
        y_test= y_hard.iloc[test_index]
#         print('#'*10,'{0}th CV'.format(n_iter),'#'*10)
        X_train = np.array(X_train)
        y_train = np.array(y_train)
        y_train = y_train.astype(float)
        X_test = np.array(X_test)
        y_test = np.array(y_test)
        y_test = y_test.astype(float)

        focal_model.compile(loss='BinaryFocalCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.005), metrics=['accuracy'])
        history = focal_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#         plt.plot(history.history['loss'], label='loss')
#         plt.ylim([0, 1])
#         plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#         plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#         plt.title("Cost Function",fontweight="bold",fontsize = 20)
#         plt.legend()
#         plt.show()
        predicted = np.round(focal_model.predict(X_test, verbose=0))
        list_acc.append(metrics.accuracy_score(y_test, predicted))
        list_pre.append(metrics.precision_score(y_test, predicted))
        list_rec.append(metrics.recall_score(y_test, predicted))
        list_f1.append(metrics.f1_score(y_test, predicted))
        list_rauc.append(metrics.roc_auc_score(y_test, predicted))
    res['Focal'] = [np.mean(list_acc), np.mean(list_pre), np.mean(list_rec), np.mean(list_f1), np.mean(list_rauc)]
    print([np.mean(list_acc), np.mean(list_pre), np.mean(list_rec), np.mean(list_f1), np.mean(list_rauc)])
    
    B = [0.00, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]  # SLS with LS
    c = 0.1  # criterion decides easy/hard
    for b in B:
        print('#'*50,'SLS',b,'#'*50)
        y_005 = []
        for i in range(len(y_hard)):
            if list(y_hard[0])[i] == 0:
                if prob_label[i] <= c:
                    y_005.append(b)  # easy sample
                else:
                    y_005.append(0) # (or 0-b) hard sample
            if list(y_hard[0])[i] == 1:
                if prob_label[i] >= 1-c:
                    y_005.append(1-b)  # easy sample
                else:
                    y_005.append(1) # (or 1+b) hard sample
        y_005 = pd.DataFrame(y_005)     

        bce005_acc = []
        bce005_pre = []
        bce005_rec = []
        bce005_f1 = []
        bce005_rocauc = []
        model_005 = create_model()
#             early_stopping = EarlyStopping(monitor='val_loss', patience=5)
        n_iter = 0
        for train_index, test_index in skf.split(X, y_hard):  # straticiation by y_hard(binary label)
            n_iter += 1
            X_train = X[train_index]
            y_005_train= y_005.iloc[train_index]
            if n_iter == 1:
                print(y_005_train.value_counts())
            X_test = X[test_index]
            y_test= y_hard.iloc[test_index]  # test with real(actual) label y
#                 print('#'*10,'{0}th CV'.format(n_iter),'#'*10)
            X_train = np.array(X_train)
            y_005_train = np.array(y_005_train)
            y_005_train = y_005_train.astype(float)
            X_test = np.array(X_test)
            y_test = np.array(y_test)
            y_test = y_test.astype(float)

            # MLP_BCE(y_005)
            model_005.compile(loss='BinaryCrossentropy', optimizer=optimizers.Adam(learning_rate = 0.005), metrics=['accuracy'])
            history = model_005.fit(X_train, y_005_train, validation_data=(X_test, y_test), epochs=epochs, verbose=0, batch_size=batch)#, callbacks=[early_stopping])
#             plt.plot(history.history['loss'], label='loss')
#             plt.ylim([0, 1])
#             plt.xlabel('Iteration',fontweight="bold",fontsize = 15)
#             plt.ylabel('Loss',fontweight="bold",fontsize = 15)
#             plt.title("Cost Function",fontweight="bold",fontsize = 20)
#             plt.legend()
#             plt.show()
            predicted = np.round(model_005.predict(X_test, verbose=0))
            bce005_acc.append(metrics.accuracy_score(y_test, predicted))
            bce005_pre.append(metrics.precision_score(y_test, predicted))
            bce005_rec.append(metrics.recall_score(y_test, predicted))
            bce005_f1.append(metrics.f1_score(y_test, predicted))
            bce005_rocauc.append(metrics.roc_auc_score(y_test, predicted))
        res['SLS({})'.format(b)] = [np.mean(bce005_acc), np.mean(bce005_pre), np.mean(bce005_rec), np.mean(bce005_f1), np.mean(bce005_rocauc)]
        print([np.mean(bce005_acc), np.mean(bce005_pre), np.mean(bce005_rec), np.mean(bce005_f1), np.mean(bce005_rocauc)])        
    res.to_csv("RNN_IMDB_5CV(SLS_opt#1_c0.1).csv", mode = 'a', float_format='%.4g')

In [ ]:
res = pd.read_csv("RNN_IMDB_5CV(SLS_opt#1_c0.1).csv")
res = res.dropna(axis=0)
res = res.rename(columns={'SLS(0.0)':'Hard'})
df_acc = res[res.iloc[:,0] == 'Acc']
df_acc = df_acc.reset_index(drop=True)
df_acc = df_acc.iloc[:,1:].astype(float)
col_name = df_acc.columns
ave = []
std = []
for i in range(12):
    ave.append(np.mean(list(df_acc.iloc[:,i])))
    std.append(np.std(list(df_acc.iloc[:,i])))
final = pd.DataFrame(ave, index=col_name, columns=["mean"])
final['std'] = std
final.sort_values("mean", ascending=False)

In [ ]:
len(df_acc)